## Calcular longitudes de rutas de parques nacionales

In [ ]:
pip install geopy

In [7]:
import os
import xml.etree.ElementTree as ET
from geopy.distance import geodesic
import csv

In [8]:
def extraer_coords(coordenadas_str):
    puntos = []
    for par in coordenadas_str.strip().split():
        lon, lat, *_ = map(float, par.split(','))
        puntos.append((lat, lon))
    return puntos

def calcular_longitud(coords):
    distancia_total = 0.0
    for i in range(len(coords) - 1):
        distancia_total += geodesic(coords[i], coords[i+1]).kilometers
    return distancia_total

def procesar_kml(filepath):
    rutas = []
    try:
        tree = ET.parse(filepath)
        root = tree.getroot()
        ns = {'kml': 'http://www.opengis.net/kml/2.2'}

        for placemark in root.findall('.//kml:Placemark', ns):
            nombre = placemark.find('kml:name', ns)
            nombre_ruta = nombre.text if nombre is not None else os.path.basename(filepath)

            coords_elem = placemark.find('.//kml:LineString/kml:coordinates', ns)
            if coords_elem is not None:
                coords = extraer_coords(coords_elem.text)
                longitud = calcular_longitud(coords)
                rutas.append((nombre_ruta, longitud))
    except Exception as e:
        print(f"Error procesando {filepath}: {e}")
    return rutas

def recorrer_carpeta(raiz):
    resultados = []
    for carpeta, _, archivos in os.walk(raiz):
        for archivo in archivos:
            if archivo.endswith('.kml'):
                filepath = os.path.join(carpeta, archivo)
                rutas = procesar_kml(filepath)
                resultados.extend(rutas)
    return resultados

def guardar_en_csv(datos, nombre_archivo):
    with open(nombre_archivo, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Nombre', 'Longitud_km'])
        for nombre, longitud in datos:
            writer.writerow([nombre, f"{longitud:.2f}"])


In [ ]:
carpeta_principal = "../../1. Obtención Y Procesado de Datos/Rutas"
resultados = recorrer_carpeta(carpeta_principal)

# Guardar resultados
guardar_en_csv(resultados, "longitudes_rutas.csv")

## Rellenar información rutas con longitudes faltantes

In [ ]:
pip install pandas openpyxl

In [ ]:
import pandas as pd

# 1. Cargar Excel con rutas
df_excel = pd.read_excel('Rutas_longitud.xlsx')

# 2. Contar filas sin longitud
faltantes = df_excel['Longitud'].isna().sum()
print(f"Filas sin longitud: {faltantes}")

# 3. Cargar CSV generado con longitudes calculadas
df_csv = pd.read_csv("longitudes_rutas.csv")

# 4. Hacer merge por nombre de ruta
df_actualizado = df_excel.copy()
df_actualizado = df_actualizado.merge(
    df_csv.rename(columns={'Nombre': 'Nombre_Ruta', 'Longitud_km': 'Longitud_csv'}),
    on='Nombre_Ruta',
    how='left'
)

# 5. Rellenar longitudes faltantes con las del CSV
df_actualizado['Longitud'] = df_actualizado['Longitud'].fillna(df_actualizado['Longitud_csv'])

# 6. Eliminar columna auxiliar
df_actualizado = df_actualizado.drop(columns=['Longitud_csv'])

# 7. Guardar el nuevo archivo
df_actualizado.to_excel('longitudes\Rutas_longitudes.xlsx', index=False)
print("Archivo actualizado guardado.")


Filas sin longitud: 310
Archivo actualizado guardado.
